In [9]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
import torch.optim 
import numpy as np #to load dataset
import math

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device in use: {device}")

Device in use: cpu


In [29]:
# define the dataset
# 0 corresponds to rotating
# 1 corresponds to grasping

# rotating
# Legnth of dataset

# num rows for one grasp
block_size = 2 * 24

#linux command
# %cd /content/drive/MyDrive/PyTorch/
unedited_rotating_dataset = np.loadtxt('rotating.csv', delimiter=",")
# Split into multiple movements over time ("block_size" seconds)
edited_rotating_dataset = unedited_rotating_dataset[:-1,1:21] #corresponding to cols 2 - 21 inclusive, excluding the last row of zeros
# Define a new shape of x
# First dimension, number of rows / (block_size * 24), then take the floor
n_blocks = math.floor(unedited_rotating_dataset.shape[0] / block_size)
# Get number of rows
rows = edited_rotating_dataset.shape[0]
X = np.zeros((rows - block_size, block_size * 20))  # (Block size * 24) seconds times 20 rows

print(X.shape)
i = 0
print(np.ndarray.flatten(edited_rotating_dataset[i * block_size : (i + 1) * block_size, :]).shape)
# For each block
for i in range(0, rows - block_size):
  # Get entire row, 1st block_size * 24 rows, 2nd of them and so on..
  X[i] = np.ndarray.flatten(edited_rotating_dataset[i * block_size : (i + 1) * block_size, :])

# X now contains mutliple blocks
y = np.zeros((rows - block_size))


# grasping
unedited_grasping_dataset = np.loadtxt('grasping.csv', delimiter=",")
edited_grasping_dataset = unedited_grasping_dataset[:-1,1:21]
# Define a new shape of x
# First dimension, number of rows / (block_size * 24), then take the floor
n_blocks = math.floor(edited_grasping_dataset.shape[0] / block_size)
# Get number of rows
rows = edited_grasping_dataset.shape[0]
X2 = np.zeros((rows - n_blocks, block_size * 20))
# For each block
for i in range(0, rows - block_size):
  # Get entire row, 1st, block_size * 24 rows, second of them and so on..
  X2[i] = np.ndarray.flatten(edited_grasping_dataset[i * block_size : (i + 1) * block_size, :])
# X now contains mutliple blocks
y2 = np.ones(n_blocks * block_size)
combined_X = np.concatenate((X, X2), axis=0)
combined_y = np.concatenate((y, y2), axis=0)

(1823, 960)
(960,)


ValueError: could not broadcast input array from shape (940,) into shape (960,)

In [6]:
# covnert into tensors since numpy uses 64 bit floating point
X = torch.tensor(combined_X, dtype=torch.float32)
y = torch.tensor(combined_y, dtype=torch.float32)
y = torch.reshape(y,(-1,1)) # a single dimension uses -1 to infer the length. new shape

print(X.shape, y.shape)

torch.Size([24, 2880]) torch.Size([1597, 1])


In [7]:
#define a class

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2880, 512), # input
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2880, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [9]:
#preparation for training

In [11]:
loss_fn = nn.BCELoss() # used for binary classification problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# train the model

n_epochs = 15
batch_size = 4 # each batch is a block

In [15]:
for epoch in range(n_epochs):
  for i in range(0, len(X), batch_size):
    Xbatch = X[i:i+batch_size] # specifying the rows
    y_pred = model(Xbatch.cuda()) # scalar
    ybatch = y[i:i+batch_size].cuda()
    loss = loss_fn(y_pred, ybatch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 14, latest loss 0.0005309406551532447


In [ ]:
# evaluate the accuracy of the model

with torch.no_grad(): # no_grad to avoid differentiating
    y_pred = model(X)
 
# rounds prediction to nearest integer, check if equal to y, 
accuracy = (y_pred.round() == y).float().mean() 
print(f"Accuracy {accuracy}")